# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.data.dataset_factory import FileDatasetFactory
from azureml.core import Workspace, Experiment, Dataset
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from automl_data_utils import get_image_data_as_df

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'covid-pneumonia-automl'

experiment=Experiment(ws, experiment_name)

compute_name = "compute-cluster"
vm_size = "STANDARD_NC6"

try:
    compute_cluster = ComputeTarget(workspace=ws, name=compute_name)
    print("Found existing cluster, using it!")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size, max_nodes=6)
    compute_cluster = ComputeTarget.create(ws, compute_name, compute_config)
compute_cluster.wait_for_completion()

Found existing cluster, using it!


In [3]:
train_data, test_data = get_image_data_as_df(ws, './Covid19-dataset/train', './Covid19-dataset/test',"covid-pneumonia-dataset-table")

Found 251 images belonging to 3 classes.
image shape: (16, 130, 100, 1)
label shape: (16,)

image shape: (16, 130, 100, 1)
label shape: (16,)

image shape: (16, 130, 100, 1)
label shape: (16,)

image shape: (16, 130, 100, 1)
label shape: (16,)

image shape: (16, 130, 100, 1)
label shape: (16,)

image shape: (16, 130, 100, 1)
label shape: (16,)

image shape: (16, 130, 100, 1)
label shape: (16,)

image shape: (16, 130, 100, 1)
label shape: (16,)

image shape: (16, 130, 100, 1)
label shape: (16,)

image shape: (16, 130, 100, 1)
label shape: (16,)

image shape: (16, 130, 100, 1)
label shape: (16,)

image shape: (16, 130, 100, 1)
label shape: (16,)

image shape: (16, 130, 100, 1)
label shape: (16,)

image shape: (16, 130, 100, 1)
label shape: (16,)

image shape: (16, 130, 100, 1)
label shape: (16,)

image shape: (11, 130, 100, 1)
label shape: (11,)

flattened shape: (251, 13001)
Found 66 images belonging to 3 classes.
image shape: (16, 130, 100, 1)
label shape: (16,)

image shape: (16, 130,

In [4]:
train_data.head()

,0,1,2,3,4,5,6,7,8,9,...,12991,12992,12993,12994,12995,12996,12997,12998,12999,class
0,0.000369,0.000369,0.000369,0.000369,0.000369,0.000369,0.000369,0.000369,0.000369,0.000369,...,0.000200,0.000246,0.000292,0.000400,0.000292,0.000369,0.000369,0.000369,0.000369,1
1,0.000123,0.000123,0.000138,0.000108,0.000092,0.000108,0.000108,0.000108,0.000092,0.000123,...,0.003091,0.002814,0.002568,0.002430,0.001984,0.001707,0.000000,0.000000,0.000000,0
2,0.000277,0.000246,0.000277,0.000308,0.000308,0.000323,0.000323,0.000338,0.000308,0.000308,...,0.000584,0.000830,0.000984,0.001200,0.001446,0.001553,0.001646,0.001738,0.001784,0
3,0.001046,0.000507,0.000062,0.000569,0.000692,0.000769,0.000830,0.001000,0.001030,0.001261,...,0.000461,0.000277,0.000277,0.000277,0.000277,0.000461,0.000000,0.000000,0.000000,1
4,0.000277,0.000354,0.000431,0.000461,0.000477,0.000446,0.000461,0.000400,0.000369,0.000246,...,0.002322,0.001599,0.001276,0.000446,0.000123,0.000031,0.000000,0.000000,0.000000,1


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [8]:
# TODO: Put your automl settings here
automl_settings = {"enable_dnn":True,
                   "experiment_timeout_minutes":60,
                   "max_cores_per_iteration":-1,
                   "enable_tf":False,
                   "featurization":"off"}

# TODO: Put your automl config here
automl_config = AutoMLConfig(task='classification',
                             primary_metric='accuracy',
                             compute_target=compute_cluster,
                             training_data=train_data,
                             label_column_name='class',
                             **automl_settings
                             )

In [9]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

ConfigException: ConfigException:
	Message: Input of type '<class 'pandas.core.frame.DataFrame'>' is not supported. Supported types: [azureml.data.tabular_dataset.TabularDataset]
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Input of type '<class 'pandas.core.frame.DataFrame'>' is not supported. Supported types: [azureml.data.tabular_dataset.TabularDataset]",
        "details_uri": "https://aka.ms/AutoMLConfig",
        "target": "training_data",
        "inner_error": {
            "code": "BadArgument",
            "inner_error": {
                "code": "ArgumentInvalid",
                "inner_error": {
                    "code": "InvalidInputDatatype"
                }
            }
        }
    }
}

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
RunDetails(remote_run).show()
remote_run.wait_for_completion()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

{'runId': 'AutoML_cb987fc2-14fd-403d-aff1-17ee0439750b',
 'target': 'compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-02-07T12:26:38.791628Z',
 'endTimeUtc': '2021-02-07T14:07:33.612458Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'compute-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"covid-pneumonia-automl","subscription_id":"510b94ba-e453-4417-988b-fbdc37b55ca7","resource_group":"aml-quickstarts-137826","workspace_name":"quick-starts-ws-137826","region":"southcentralus","compute_target":"compute-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":null,"y_min":null,"y_max":n

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [11]:
#TODO: Save the best model
best_run = remote_run.get_best_child()
best_run_metrics = best_run.get_metrics()
best_run_parameters = best_run.get_details()
best_run_properties = best_run_parameters['properties']

print("Best Run:",best_run.id)
print("Algorithm:", best_run_properties['run_algorithm'])
print("Metric:", best_run_properties['primary_metric'])
print("Best Score:",best_run_properties['score'])


Best Run: AutoML_cb987fc2-14fd-403d-aff1-17ee0439750b_14
Algorithm: VotingEnsemble
Metric: accuracy
Best Score: 0.9400000000000001


In [12]:
best_run.get_file_names()

['accuracy_table',
 'automl_driver.py',
 'azureml-logs/55_azureml-execution-tvmps_2ab0a904e4a8270e2643bc41efbfe119f35c445296c019fd538f1ed574fc37ec_d.txt',
 'azureml-logs/65_job_prep-tvmps_2ab0a904e4a8270e2643bc41efbfe119f35c445296c019fd538f1ed574fc37ec_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_2ab0a904e4a8270e2643bc41efbfe119f35c445296c019fd538f1ed574fc37ec_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'confusion_matrix',
 'logs/azureml/87_azureml.log',
 'logs/azureml/azureml_automl.log',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log',
 'outputs/conda_env_v_1_0_0.yml',
 'outputs/env_dependencies.json',
 'outputs/internal_cross_validated_models.pkl',
 'outputs/model.pkl',
 'outputs/pipeline_graph.json',
 'outputs/scoring_file_v_1_0_0.py']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [13]:
registered_model = best_run.register_model(model_name='covid-pneumonia-automl',
                                            model_path='./outputs/model.pkl')


TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service